In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn

import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2015-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2015-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.351213e+09  1.325373e+09  4.608670e+08  5.710782e+08   
1 2001-03-31  1.179955e+09  1.187259e+09  6.161152e+08  7.159990e+08   
2 2001-03-31  9.991729e+08  9.956328e+08  4.859796e+08  2.470630e+08   
3 2001-03-31  7.287602e+08  7.165592e+08  4.586744e+08  4.985467e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.527046e+08  8.237827e+08        NaT  9.127740   9.127740  5.470572   
1  1.100090e+08  8.260080e+08 2009-01-16  7.970853  17.098593  7.313395   
2  4.301490e+08  6.772120e+08 2008-11-23  6.749630  23.848224  5.768663   
3  5.936475e+07  5.579114e+08        NaT  4.922934  28.771157  5.444545   
4  2.469325e+07  2.951088e+08        NaT  2.983418  31.754575  3.022585   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.470572  6.778798   6.778798  
1  12.783967  8.499033  15.277831  
2  18.552629  2.932681  18.210512  
3  23.997174  5.917837  24.128348  
4  27.019759  3.209880  27.338229

In [3]:
len(rk)

10762

In [33]:
bhc = pd.read_csv("INSTITUTIONS2_201604.csv", encoding='latin-1')
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   
2       223         0         0 ...               Portland-South Portland, ME   
3       223         0         0 ...                                       NaN   
4       223         0         0 ...               Portland-South Portland, ME   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   
2    38860               Portland-South Portland, ME      38860   
3        0                                       NaN          0   
4    38860               Portland-South Portland, ME      38860   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  
2              1              0       NaN            0            0 NaN  
3              0              0       NaN            0            0 NaN  
4              1              0       NaN            0            0 NaN  

[5 rows x 116 columns]

In [34]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [35]:
len(bhc)

27594

In [36]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  

[2 rows x 116 columns]

In [37]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
len(bhc2)

27594

In [38]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [39]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635                           AuburnBank  AL      SM     10.0   
19    39  430036            Robertson Banking Company  AL      NM      6.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      2.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [44]:
bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]

Empty DataFrame
Columns: [Cert, IDRSSD, Name, ST, BkClass, Offices, Parent, ParRSSD, ParST, UltCert]
Index: []

In [49]:
bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [50]:
## extract ticker info from prior listing:
rkh = pd.read_csv("BkAvgAstPR_FailDt_BHC_BkClass_ult2.csv", index_col=0)

len(rkh)

10761

In [52]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','Ticker']]
len(rkh2)

7031

In [78]:
rkh2.tail(2)

Rank    ParRSSD Ticker
10736  10737  1039502.0    JPM
10738  10739  1098648.0   CASS

In [81]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3)

407

In [82]:
rkh3.head()

Rank
ParRSSD   Ticker      
1039502.0 JPM        1
1073757.0 BAC        2
1951350.0 C          3
1120754.0 WFC        4
1119794.0 USB        6

In [83]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank
0  1039502.0    JPM     1
1  1073757.0    BAC     2
2  1951350.0      C     3
3  1120754.0    WFC     4
4  1119794.0    USB     6

In [84]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABYB          1
ACNB          1
ALBY          1
ALLY          1

In [85]:
rkh5[rkh5['ParRSSD']>1]

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [88]:
bhc2.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [90]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [91]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4820.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [93]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4820.0  Bank Of America Corp  1073757.0    NC     3510

In [94]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eTLcs    pr_eDD  \
0  1.351213e+09  1.325373e+09  4.608670e+08   ...      5.470572  6.778798   
1  1.179955e+09  1.187259e+09  6.161152e+08   ...     12.783967  8.499033   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   6.778798   628.0        N   5547.0  Jpmorgan Chase & Co.  1039502.0   
1  15.277831  3510.0        N   4820.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [95]:
rkh4.head(2)

ParRSSD Ticker  Rank
0  1039502.0    JPM     1
1  1073757.0    BAC     2

In [96]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eDD   pr_eDDcs  \
0  1.351213e+09  1.325373e+09  4.608670e+08   ...    6.778798   6.778798   
1  1.179955e+09  1.187259e+09  6.161152e+08   ...    8.499033  15.277831   

     Cert BkClass  Offices                Parent    ParRSSD  ParST  UltCert  \
0   628.0       N   5547.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0   
1  3510.0       N   4820.0  Bank Of America Corp  1073757.0     NC   3510.0   

   Ticker  
0     JPM  
1     BAC  

[2 rows x 28 columns]

In [99]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [100]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices    ...           aeTotLn        aeDDep  \
0        NaT       N   5547.0    ...      4.608670e+08  5.710782e+08   
1 2009-01-16       N   4820.0    ...      6.161152e+08  7.159990e+08   

         aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
0  2.527046e+08  8.237827e+08  9.127740   9.127740  5.470572   5.470572   
1  1.100090e+08  8.260080e+08  7.970853  17.098593  7.313395  12.783967   

     pr_eDD   pr_eDDcs  
0  6.778798   6.778798  
1  8.499033  15.277831  

[2 rows x 27 columns]

In [ ]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist2015.csv")

In [102]:
len(rk4)

10762

In [106]:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm)

212

In [107]:
rkm.head()

Rank   IDRSSD      xAsOf                            xName xState  \
46     47  1456501 2015-12-31        MORGAN STANLEY BANK, N.A.     UT   
64     65  3212149 2015-12-31                     UBS BANK USA     UT   
91     92  2489805 2015-12-31  MORGAN STANLEY PRIVATE BANK, NA     NY   
148   149   527954 2015-12-31     FIRST NATIONAL BANK OF OMAHA     NE   
150   151   311845 2015-12-31                      ARVEST BANK     AR   

       xCert      nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
46   32992.0 2001-03-31      NaT       N      1.0    ...      1.474818e+07   
64   57565.0 2003-09-30      NaT      NM      1.0    ...      1.809031e+07   
91   34221.0 2010-09-30      NaT       N      2.0    ...      1.095995e+07   
148   5452.0 2001-03-31      NaT       N    124.0    ...      6.612029e+06   
150   8728.0 2001-03-31      NaT      SM    273.0    ...      6.254917e+06   

           aeDDep       aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
46   3.941939e+07     0.000000  3.941939e+07  0.323387  59.708383  0.175063   
64   2.752866e+07     0.000000  2.752866e+07  0.209973  64.470590  0.214735   
91   1.657527e+07     0.000000  1.657527e+07  0.128640  68.724863  0.130096   
148  7.861507e+06     0.000000  7.861507e+06  0.067805  74.058541  0.078486   
150  8.130339e+06  3835.733333  8.134175e+06  0.065529  74.190706  0.074247   

      pr_eTLcs    pr_eDD   pr_eDDcs  
46   54.043349  0.467915  55.185467  
64   59.776600  0.326770  60.224352  
91   64.650497  0.196751  65.006078  
148  70.478436  0.093317  71.226246  
150  70.632486  0.096509  71.405867  

[5 rows x 27 columns]

In [108]:
rkm.to_csv('rk_missingtickers.csv')

In [ ]:
## filled in many missing tickers

In [111]:
rkm2 = pd.read_csv('rk_missingtickers2.csv',index_col=0)
rkm2.head()

Rank   IDRSSD       xAsOf                            xName xState  xCert  \
46     47  1456501  12/31/2015        MORGAN STANLEY BANK, N.A.     UT  32992   
64     65  3212149  12/31/2015                     UBS BANK USA     UT  57565   
91     92  2489805  12/31/2015  MORGAN STANLEY PRIVATE BANK, NA     NY  34221   
148   149   527954  12/31/2015     FIRST NATIONAL BANK OF OMAHA     NE   5452   
150   151   311845  12/31/2015                      ARVEST BANK     AR   8728   

         nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
46   3/31/2001      NaN       N        1    ...      1.474818e+07   
64   9/30/2003      NaN      NM        1    ...      1.809031e+07   
91   9/30/2010      NaN       N        2    ...      1.095995e+07   
148  3/31/2001      NaN       N      124    ...      6.612029e+06   
150  3/31/2001      NaN      SM      273    ...      6.254917e+06   

           aeDDep       aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
46   3.941939e+07     0.000000  3.941939e+07  0.323387  59.708383  0.175063   
64   2.752866e+07     0.000000  2.752866e+07  0.209973  64.470591  0.214735   
91   1.657527e+07     0.000000  1.657527e+07  0.128640  68.724863  0.130096   
148  7.861507e+06     0.000000  7.861507e+06  0.067805  74.058541  0.078486   
150  8.130339e+06  3835.733333  8.134175e+06  0.065529  74.190706  0.074247   

      pr_eTLcs    pr_eDD   pr_eDDcs  
46   54.043349  0.467915  55.185467  
64   59.776600  0.326770  60.224352  
91   64.650497  0.196751  65.006078  
148  70.478436  0.093317  71.226246  
150  70.632486  0.096509  71.405867  

[5 rows x 27 columns]

In [114]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
46   2162966     MS
64   4795461    UBS
91   2162966     MS
161  2747644   UMPQ
167  1240737  ISDAF

In [117]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p)

(39, 37)

In [121]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
46                            MORGAN STANLEY BANK, N.A.   
64                                         UBS BANK USA   
91                      MORGAN STANLEY PRIVATE BANK, NA   
161                                         UMPQUA BANK   
167                    ISRAEL DISCOUNT BANK OF NEW YORK   
242                                      BANK LEUMI USA   
258                                  GREAT WESTERN BANK   
319                                     BENEFICIAL BANK   
354                                      BROOKLINE BANK   
381                            SABADELL UNITED BANK, NA   
488                                      HOMETRUST BANK   
510                                      FIRST NBC BANK   
522                                        CENTIER BANK   
588                                           TOTALBANK   
623                                 INTER NATIONAL BANK   
637                                   BANK RHODE ISLAND   
639                                   ESSA BANK & TRUST   
640                      HEARTLAND BANK & TRUST COMPANY   
657                                    PULASKI BANK, NA   
735                                   HOME FEDERAL BANK   
756                                    INDEPENDENT BANK   
758                                       HERITAGE BANK   
761                                       HOME BANK, NA   
796                                     GRANDPOINT BANK   
853                            REPUBLIC BANK OF CHICAGO   
869                                     BLUE HILLS BANK   
877                                        BANK OF GUAM   
969                                  WOORI AMERICA BANK   
994                               SECURITY FEDERAL BANK   
1004                                       ENTEGRA BANK   
1007                                          CAPE BANK   
1027                      INDEPENDENCE BANK OF KENTUCKY   
1033  FIRST FEDERAL SAVINGS AND LOAN ASSOCIATION OF ...   
1049  FIRST NATIONAL BANK TEXAS DBA FIRST CONVENIENC...   
1109                      ADIRONDACK TRUST COMPANY, THE   
1140                                    ALLEGIANCE BANK   
1154                                     GREEN BANK, NA   
1189                              UNITED COMMUNITY BANK   
1195                                HAMILTON STATE BANK   
1598                                    MERCANTILE BANK   
2663                              AMERICAN MIDWEST BANK   
2706                                 FIRST IPSWICH BANK   
3877                            FORT HOOD NATIONAL BANK   
4669                                MARINE BANK & TRUST   
7606                            BROWN COUNTY STATE BANK   

                                        Parent Ticker  
46                              Morgan Stanley    NaN  
64                                Ubs Group Ag    NaN  
91                              Morgan Stanley    NaN  
161                       Umpqua Holdings Corp    NaN  
167               Israel Discount Bank Limited    NaN  
242                  Bank Leumi Le-Israel B.M.    NaN  
258                  Great Western Bancorp Inc    NaN  
319                     Beneficial Bancorp Inc    NaN  
354                      Brookline Bancorp Inc    NaN  
381                    Banco De Sabadell, S.A.    NaN  
488                   Hometrust Bancshares Inc    NaN  
510                          First Nbc Bank HC    NaN  
522                       First Bancshares Inc    NaN  
588                Banco Popular Espanol, S.A.    NaN  
623   Grupo Financiero Banorte, S.A.b. De C.V.    NaN  
637                      Brookline Bancorp Inc    NaN  
639                           Essa Bancorp Inc    NaN  
640                      Heartland Bancorp Inc    NaN  
657                          Pulaski Fin Corp.    NaN  
735                               Hf Fin Corp.    NaN  
756                 Independent Bank Group Inc    NaN  
758                          Heritage Fin Corp    NaN  
761                           Home Bancorp Inc 

In [126]:
rk4c = rk4.copy()


In [128]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1097182: 'UCBA',
 1108097: 'FCBC',
 1118854: 'IEBS',
 1201194: 'RBCAA',
 1204560: 'FBMS',
 1208120: 'HLAN',
 1232536: 'BLMIF',
 1240737: 'ISDAF',
 1242834: 'WF',
 1253773: 'SFDL',
 1981384: 'BNDS',
 2162966: 'MS',
 2166124: 'HFWA',
 2427665: 'ENFC',
 2500700: 'HFFC',
 2631510: 'BRKL',
 2747644: 'UMPQ',
 3138267: 'ADKT',
 3140288: 'IBTX',
 3369995: 'BPESY',
 3392331: 'HMBH',
 3407020: 'GBOOY',
 3474835: 'GNBC',
 3485541: 'FNBC',
 3742020: 'BHBK',
 3744239: 'ABTX',
 3811832: 'GPNC',
 3832819: 'CBNJ',
 3832828: 'PULB',
 3846629: 'BNCL',
 3851191: 'HBCP',
 3854268: 'ESSA',
 4272869: 'BKGMF',
 4366003: 'HTBI',
 4438366: 'FNWB',
 4795461: 'UBS',
 4809920: 'GWB'}

In [132]:
tdct[4809920]

'GWB'

In [140]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [141]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
46                            MORGAN STANLEY BANK, N.A.   
64                                         UBS BANK USA   
91                      MORGAN STANLEY PRIVATE BANK, NA   
161                                         UMPQUA BANK   
167                    ISRAEL DISCOUNT BANK OF NEW YORK   
242                                      BANK LEUMI USA   
258                                  GREAT WESTERN BANK   
319                                     BENEFICIAL BANK   
354                                      BROOKLINE BANK   
381                            SABADELL UNITED BANK, NA   
488                                      HOMETRUST BANK   
510                                      FIRST NBC BANK   
522                                        CENTIER BANK   
588                                           TOTALBANK   
623                                 INTER NATIONAL BANK   
637                                   BANK RHODE ISLAND   
639                                   ESSA BANK & TRUST   
640                      HEARTLAND BANK & TRUST COMPANY   
657                                    PULASKI BANK, NA   
735                                   HOME FEDERAL BANK   
756                                    INDEPENDENT BANK   
758                                       HERITAGE BANK   
761                                       HOME BANK, NA   
796                                     GRANDPOINT BANK   
853                            REPUBLIC BANK OF CHICAGO   
869                                     BLUE HILLS BANK   
877                                        BANK OF GUAM   
969                                  WOORI AMERICA BANK   
994                               SECURITY FEDERAL BANK   
1004                                       ENTEGRA BANK   
1007                                          CAPE BANK   
1027                      INDEPENDENCE BANK OF KENTUCKY   
1033  FIRST FEDERAL SAVINGS AND LOAN ASSOCIATION OF ...   
1049  FIRST NATIONAL BANK TEXAS DBA FIRST CONVENIENC...   
1109                      ADIRONDACK TRUST COMPANY, THE   
1140                                    ALLEGIANCE BANK   
1154                                     GREEN BANK, NA   
1189                              UNITED COMMUNITY BANK   
1195                                HAMILTON STATE BANK   
1598                                    MERCANTILE BANK   
2663                              AMERICAN MIDWEST BANK   
2706                                 FIRST IPSWICH BANK   
3877                            FORT HOOD NATIONAL BANK   
4669                                MARINE BANK & TRUST   
7606                            BROWN COUNTY STATE BANK   

                                        Parent Ticker  
46                              Morgan Stanley     MS  
64                                Ubs Group Ag    UBS  
91                              Morgan Stanley     MS  
161                       Umpqua Holdings Corp   UMPQ  
167               Israel Discount Bank Limited  ISDAF  
242                  Bank Leumi Le-Israel B.M.  BLMIF  
258                  Great Western Bancorp Inc    GWB  
319                     Beneficial Bancorp Inc   BNCL  
354                      Brookline Bancorp Inc   BRKL  
381                    Banco De Sabadell, S.A.   BNDS  
488                   Hometrust Bancshares Inc   HTBI  
510                          First Nbc Bank HC   FNBC  
522                       First Bancshares Inc   FBMS  
588                Banco Popular Espanol, S.A.  BPESY  
623   Grupo Financiero Banorte, S.A.b. De C.V.  GBOOY  
637                      Brookline Bancorp Inc   BRKL  
639                           Essa Bancorp Inc   ESSA  
640                      Heartland Bancorp Inc   HLAN  
657                          Pulaski Fin Corp.   PULB  
735                               Hf Fin Corp.   HFFC  
756                 Independent Bank Group Inc   IBTX  
758                          Heritage Fin Corp   HFWA  
761                           Home Bancorp Inc 

In [143]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' '])

(537, 492)

In [144]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist2015.csv")

In [146]:
## process ultcert version:

In [147]:
rk5 = rk4.copy()

In [148]:
ult = set(rk5['UltCert'])
len(ult)

6437

In [165]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [166]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc)

4648

In [167]:
len(bhcc[bhcc['Ticker']>''])

532

In [168]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [169]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
39     16571.0                 Bank Of Montreal    BMO  1231333.0     0
221     4999.0              Cadence Bancorp Llc    NaN  4037349.0    TX
251    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
590    26342.0             Royal Bank Of Canada     RY  1232497.0     0
636     4999.0              Cadence Bancorp Llc    NaN  4037349.0    TX
929    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
949    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
1150    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1169   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1549    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
1622   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1653   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1715      89.0   Community First Bancshares Inc    NaN  2754585.0    AR
1828    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
2043   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2477   26342.0             Royal Bank Of Canada     RY  1232497.0     0
2836   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4013   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4123   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4131   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4270   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4310    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
4978   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
5336   11256.0          Pinnacle Bancshares Inc    NaN  3451416.0    AR
5758    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6019    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
6300   11256.0          Pinnacle Bancshares Inc    NaN  3451416.0    AR
6908      89.0   Community First Bancshares Inc    NaN  2754585.0    AR
6919   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
6924   17399.0                   First Fin Corp    NaN  1067514.0    KS
7456   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7612    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
7751   15880.0       First National Bancorp Inc    NaN  2619349.0    AR
7833   15880.0       First National Bancorp Inc    NaN  2619349.0    AR
8083    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
9020   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9317    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
9591    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9723   17399.0                   First Fin Corp    NaN  1067514.0    KS
10150  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10194  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10494  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [170]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6924  17399.0  First Fin Corp    NaN  1067514.0    KS
9723  17399.0  First Fin Corp    NaN  1067514.0    KS

In [171]:
len(bhcc)

4648

In [172]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2)

4606

In [173]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [174]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [175]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [177]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase & Co.'

In [ ]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [180]:
len(rk5c[rk5c['Ticker']>''])

2801

In [181]:
j

3881

In [182]:
len(rk5c)

10762

In [183]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult2015.csv")

In [191]:
rk4[rk4['Ticker']=='ZION']  ## current active affiliate

Rank  IDRSSD      xAsOf                      xName xState   xCert  \
104   105  276579 2015-12-31  ZIONS FIRST NATIONAL BANK     UT  2270.0   

         nAsOf FailDate BkClass  Offices    ...         aeTotLn        aeDDep  \
104 2001-03-31      NaT       N    480.0    ...      11248142.7  1.128952e+07   

        aeFDep      aeTotDep    pr_eTA  pr_eTAcs    pr_eTL   pr_eTLcs  \
104  1013563.5  1.230309e+07  0.107625  70.21797  0.133517  66.117973   

       pr_eDD   pr_eDDcs  
104  0.134009  66.775522  

[1 rows x 27 columns]

In [192]:
rk5c[rk5c['Ticker']=='ZION']  ## all acquired & consolidated banks

Rank   IDRSSD      xAsOf                                 xName xState  \
104    105   276579 2015-12-31             ZIONS FIRST NATIONAL BANK     UT   
142    143   837260 2015-09-30               CALIFORNIA BANK & TRUST     CA   
149    150   676656 2015-09-30                        AMEGY BANK, NA     TX   
280    281  1004368 2015-09-30              NATIONAL BANK OF ARIZONA     AZ   
310    311   456960 2015-09-30                     NEVADA STATE BANK     NV   
403    404   933957 2015-09-30              VECTRA BANK COLORADO, NA     NM   
671    672   807665 2009-06-30                     VINEYARD BANK, NA     CA   
1017  1018   458263 2006-12-31                  STOCKMEN'S BANK, THE     AZ   
1064  1065  1198641 2015-09-30  COMMERCE BANK OF WASHINGTON, NA, THE     WA   
1132  1133  2479310 2008-06-30                     SILVER STATE BANK     NV   
1524  1525   720465 2008-12-31                         ALLIANCE BANK     CA   
1591  1592   618656 2004-09-30                            KLEIN BANK     TX   
2579  2580   105950 2001-09-30          MINNEQUA BANK OF PUEBLO, THE     CO   
3037  3038   576158 2003-06-30                             MAXIMBANK     TX   
3914  3915   309664 2006-06-30                           RANCHO BANK     CA   
3996  3997   521550 2003-12-31                        LONE STAR BANK     TX   
4485  4486  2068134 2009-03-31            GREAT BASIN BANK OF NEVADA     NV   
6028  6029   626754 2001-03-31              LEAGUE CITY BANK & TRUST     TX   
6208  6209  3161993 2008-03-31                           CHOICE BANK     AZ   
6504  6505   687269 2007-06-30        INTERCONTINENTAL NATIONAL BANK     TX   
6543  6544   456670 2002-09-30                   FRONTIER STATE BANK     AZ   
8298  8299  3374186 2015-03-31          COMMERCE BANK OF OREGON, THE     OR   
9850  9851  2907448 2003-06-30               SOUTHLAND BUSINESS BANK     CA   

        xCert      nAsOf   FailDate BkClass  Offices     ...      \
104    2270.0 2001-03-31        NaT       N    480.0     ...       
142   20852.0 2001-03-31        NaT      NM     99.0     ...       
149   24107.0 2001-03-31        NaT       N     82.0     ...       
280   20626.0 2001-03-31        NaT       N     66.0     ...       
310   18113.0 2001-03-31        NaT      NM     50.0     ...       
403    2993.0 2001-03-31        NaT       N     37.0     ...       
671   23556.0 2001-03-31 2009-07-17       N     17.0     ...       
1017  23182.0 2001-03-31        NaT      NM     43.0     ...       
1064  27298.0 2001-03-31        NaT       N      2.0     ...       
1132  34194.0 2001-03-31 2008-09-05      NM     18.0     ...       
1524  23124.0 2001-03-31 2009-02-06      NM      5.0     ...       
1591  22866.0 2001-03-31        NaT      NM     27.0     ...       
2579   1773.0 2001-03-31        NaT      NM      5.0     ...       
3037   9017.0 2001-03-31        NaT      NM      9.0     ...       
3914  22934.0 2001-03-31        NaT      NM      4.0     ...       
3996  22304.0 2001-03-31        NaT      NM      6.0     ...       
4485  33824.0 2001-03-31 2009-04-17      NM      5.0     ...       
6028  17577.0 2001-03-31        NaT      NM      4.0     ...       
6208  57469.0 2003-03-31        NaT      NM      3.0     ...       
6504  19112.0 2001-03-31        NaT       N      3.0     ...       
6543  24227.0 2001-03-31        NaT      NM      7.0     ...       
8298  58223.0 2005-12-31        NaT      NM      1.0     ...       
9850  35287.0 2001-03-31        NaT      NM      1.0     ...       

           aeTotLn        aeDDep        aeFDep      aeTotDep    pr_eTA  \
104   1.124814e+07  1.128952e+07  1.013564e+06  1.230309e+07  0.107625   
142   7.675023e+06  8.519186e+06  3.219466e+04  8.551380e+06  0.069663   
149   6.722988e+06  7.001789e+06  6.462602e+05  7.648049e+06  0.066635   
280   3.325315e+06  3.496657e+06  2.862949e+03  3.499520e+06  0.028387   
310   2.461972e+06  3.219857e+06  0.000000e+00  3.219857e+06  0.024974   
403   1.905816e+06  1.916720e+06